This notebook can be used to run the simulation instead of having to type the commands into the command line interface

In [ ]:
# Copyright (c) 2021 *****************. All rights reserved.
# Licensed under the Apache License, Version 2.0, which is in the LICENSE file.

import multiprocessing as mp
import os
import time
from pathlib import Path
import generate_input_files

# Generate Model Inputs

### 1. Enter the name of the folder within the MODEL_RUNS directory that contains the files for the scenarios you want to run

In [ ]:
model_run_name = 'test'

### 2. Generate the input files

Enter the timezone for model inputs

In [ ]:
model_workspace = Path.cwd() / f'../MODEL_RUNS/{model_run_name}'

timezone = 'US/Pacific'

generate_input_files.generate_inputs(model_workspace, timezone)

# Solve Model Scenarios

### 1. Get the number of CPU cores (processors) that your computer has

In [ ]:
num_processors = mp.cpu_count()
num_processors

### 2. If you want to solve multiple scenarios in parallel, you should only run n-1 solvers, where n is the number of CPU cores on your machine.
This opens as many parallel threads as possible while leaving some processing power available on your machine to do other tasks. Each time you run the following cell, it will open a new parallel solver window.

In [ ]:
os.system(f'start cmd /k "cd ..\MODEL_RUNS\{model_run_name} & activate switch_247 & switch solve-scenarios"') 